# <center>DDPM、DDIM原理讲解与公式推导
***

# <center>前提知识
## <center>1.Markov：当前位置的概率只会受前一时刻概率影响
## <center>2.正态分布(高斯分布)叠加性
<!-- 在这里插入图片 -->
<center>
当有两个独立的正态分布变量N₁和N₂时，它们的均值与方差分别为μ₁, μ₂ 和 σ₁², σ₂²。它们的和N = N₁ + N₂ 的均值和方差可以表示如下：

\
均值：
$E(N) = E(N₁ + N₂) = \mu₁ + \mu₂$

方差：
$Var(N) = Var(N₁ + N₂) = \sigma₁² + \sigma₂²$

相减时：

均值：
$E(N) = E(N₁ + N₂) = \mu₁ - \mu₂$

方差：
$Var(N) = Var(N₁ - N₂) = \sigma₁² + \sigma₂²$

## 3.贝叶斯：
$P(A | B) = \frac{{P(B | A) \cdot P(A)}}{{P(B)}}$
## 4.在给定条件C下的贝叶斯：
$P(A | B, C) = \frac{{P(B | A, C) \cdot P(A | C)}}{{P(B | C)}}$
***


# <center>Let's start it！
<center><img src="./DDPM.png" width = "600" height = "200" alt="图片名称" align=center />\



### <center><font color=red>Forward Process</font>

<center>

$p(x_t|x_{t-1})=\mathcal{N}(x_t;\sqrt{\alpha_t}x_{t-1}, \beta_t I) $\
\
$x_t = \sqrt{\alpha_t} x_{t-1} + \sqrt{\beta_t} \epsilon_t \; \; \epsilon_t \sim \mathcal{N}(0,I) $\
\
$\alpha_t=1-\beta_t$\
\
![](1.png)\
\
$设：\bar{\alpha}_t= \alpha_1...\alpha_t$\
\
$p(x_{t}|x_0) = \sqrt{\bar{\alpha}_{t}}x_0 + \sqrt{1-\bar{\alpha}_{t}}\epsilon \; \; \epsilon \sim \mathcal{N}(0
, I) $
***
</center>

### <center><font color=red>Sampling Process</font>
<center>

$p(x_{t-1}|x_t) = \frac{{p(x_t | x_{t-1}) \cdot p(x_{t-1})}}{{p(x_t)}}$\
\
问题是：$p(x_{t-1}) \; , p(x_t)$未知，因为无法直接从$x_t$推导$x_{t-1}$\
但是可以在$x_0$的条件下进行推导\
\
$∴p(x_{t-1}|x_t, x_0) = \frac{{p(x_t | x_{t-1}, x_0) \cdot p(x_{t-1}|x_0)}}{{p(x_t|x_0)}} = \frac{{p(x_t | x_{t-1}) \cdot p(x_{t-1}|x_0)}}{{p(x_t|x_0)}}$ \
\
$∵$\
1.$p(x_{t-1}|x_0) = \sqrt{\bar{\alpha}_{t-1}}x_0 + \sqrt{1-\bar{\alpha}_{t-1}}\epsilon \sim \mathcal{N}(\sqrt{\bar{\alpha}_{t-1}}x_0, 1-\bar{\alpha}_{t-1}) $\
\
2.$p(x_{t}|x_0) = \sqrt{\bar{\alpha}_{t}}x_0 + \sqrt{1-\bar{\alpha}_{t}}\epsilon \sim \mathcal{N}(\sqrt{\bar{\alpha}_{t}}x_0, 1-\bar{\alpha}_{t}) $\
\
3.$p(x_{t}|x_{t-1}) = \sqrt{\alpha_{t}}x_{t-1} + \sqrt{1-\bar{\alpha}_{t}}\epsilon  \sim \mathcal{N}(\sqrt{\alpha_{t}}x_{t-1}, 1-\bar{\alpha}_{t}) $\
\
$p(x_{t-1}|x_t, x_0) \propto exp(-\frac{1}{2}(\frac{(x_t-\sqrt{\alpha_t}x_{t-1})^2}{\beta_t})+\frac{(x_{t-1}-\sqrt{\bar{\alpha}}_{t-1}x_0)^2}{1-\bar{\alpha}_{t-1}}-\frac{(x_t-\sqrt{\bar{\alpha}_t}x_0)^2}{1-\bar{\alpha}_t})$ \
\
$Tips\ Normal\ distribution：$ $exp(-\frac{(x-\mu)^2}{2\sigma^2})=exp(-\frac{1}{2}(\frac{1}{\sigma^2}x^2-\frac{2\mu}{\sigma^2}x+\frac{\mu^2}{\sigma^2}))$\
\
把上述式子展开，并整理成正态分布形式：\
![](4.png)\
\
$\sigma^2 = \frac{\beta_t(1-\bar{\alpha}_{t-1})}{\alpha_t(1-\bar{\alpha}_{t-1})+\beta_t}$\
\
$\mu = \frac{\sqrt{\alpha_t}(1-\bar{\alpha}_{t-1})}{1-\bar{\alpha_t}}x_t+\frac{\sqrt{\bar{\alpha}_{t-1}}\beta_t}{1-\bar{\alpha}_t}x_0$\
\
$x_0是未知的,我们可以选择一个未知的函数使得x_0=f(x_t)，但是观察之前的公式可以得知，从x_0推导x_t的公式取逆就是一条满足的公式$\
\
$x_0 = \frac{1}{\sqrt{\bar{\alpha}}_t}(x_t-\sqrt{1-\bar{\alpha}_t}\epsilon_t)$\
\
代入\
\
$\mu=\frac{1}{\sqrt{\alpha_t}}(x_t-\frac{\beta_t}{\sqrt{1-\bar\alpha}_t}\epsilon_t)$\
$因为\epsilon_t是个噪音，不知道是什么来的，所以用神经网络去拟合，表示为：\epsilon_\theta(x_t,t)$\
从而也可以得到要优化的目标，就是拟合出每一步的噪音，$||\epsilon_t-\epsilon_\theta(x_t,t)||^2$



### <center> 万事具备
<center>

最后的采样公式：\
\
$x_{t-1}=\frac{1}{\sqrt{\alpha_t}}(x_t-\frac{\beta_t}{\sqrt{1-\bar\alpha}_t}\epsilon_\theta(x_t,t))+\sigma_t\epsilon \;\; \epsilon \sim \mathcal{N}(0
, I)$
</center>

### <center>算法流程
<center>

![](5.png)
***

# <center>DDIM

<center>

DDPM存在的问题，为了公式推导的简单，它定义了前向是马尔科夫链，但是这使得反向采样时也必须是马尔可夫过程，使得每一个状态都只受前一个状态的影响，所以训练多少步，就需要采样多少步，从而速度十分慢。
### 回到最初
$p(x_{t-1}|x_t, x_0) = \frac{{p(x_t | x_{t-1}, x_0) \cdot p(x_{t-1}|x_0)}}{{p(x_t|x_0)}}$\
\
$已知：p(x_{t-1}|x_0)，{p(x_t|x_0)}，未知：p(x_t | x_{t-1}, x_0)$\
\
$假设p(x_{t-1} | x_t, x_0)为任意分布，只需要其满足等式即可$\
\
$设：p(x_{t-1} | x_t, x_0) \sim \mathcal{N}(kx_0+mx_t, \sigma^2I),$\
\
$x_{t-1}=kx_0+mx_t+\sigma\epsilon \;\; \epsilon \sim \mathcal{N}(0,I)$\
\
$∵x_t = \sqrt{\alpha_t} x_{t-1} + \sqrt{\beta_t} \epsilon_t \; \; \epsilon_t \sim \mathcal{N}(0,I) $\
\
$∴x_{t-1} = kx_0 + m(\sqrt{\bar{\alpha}_t}x_0 + \sqrt{\beta_t} \epsilon_t) + \sigma\epsilon$ \
\
合并同类项\
\
$x_{t-1} = (k+m\sqrt{\bar{\alpha}_t})x_0 + (m\sqrt{1-\bar{\alpha}_t})\epsilon_t + \sigma\epsilon$\
\
$x_{t-1} = (k+m\sqrt{\bar{\alpha}_t})x_0 + \epsilon' \;\; \epsilon' \sim \mathcal{N}(0,m^2(1-\bar{\alpha_t})+\sigma^2)$\
\
$∵x_{t-1} = \sqrt{\bar{\alpha}_{t-1}}x_0 + \sqrt{1-\bar{\alpha}_{t-1}}\epsilon$\
\
$∴k+m\sqrt{\bar{\alpha}_t}=\bar{\alpha}_{t-1}和m^2(1-\bar{\alpha_t})+\sigma^2=1-\bar{\alpha}_{t-1}$\
\
$求得：m=\frac{\sqrt{1-\bar{\alpha}_{t-1}-\sigma^2}}{\sqrt{1-\bar{\alpha}_t}}和k=\sqrt{\bar{\alpha}_{t-1}}-\sqrt{1-\bar{\alpha}_{t-1}-\sigma^2}\frac{\sqrt{\bar{\alpha}_t}}{\sqrt{1-\bar{\alpha}_t}}$\
\
$∴p(x_{t-1} | x_t, x_0) \sim \mathcal{N}((\sqrt{\bar{\alpha}_{t-1}}-\sqrt{1-\bar{\alpha}_{t-1}-\sigma^2}\frac{\sqrt{\bar{\alpha}_t}}{\sqrt{1-\bar{\alpha}_t}})x_0+\frac{\sqrt{1-\bar{\alpha}_{t-1}-\sigma^2}}{\sqrt{1-\bar{\alpha}_t}}x_t, \sigma^2I)$\
\
$\sim \mathcal{N}(\sqrt{\bar{\alpha}_{t-1}}x_0 + \sqrt{1-\bar{\alpha}_{t-1}-\sigma^2}\frac{x_t-\sqrt{\bar{\alpha}_t}x_0}{\sqrt{1-\bar{\alpha}_t}}, \sigma^2I)$\
\
$把x_0用x_t和\epsilon_\theta代替，同DDPM$\
\
$x_0 = \frac{1}{\sqrt{\bar{\alpha}}_t}(x_t-\sqrt{1-\bar{\alpha}_t}\epsilon_\theta)$\
\
$\epsilon_\theta = \frac{x_t-\sqrt{\bar{\alpha}_t}x_0}{\sqrt{1-\bar{\alpha}_t}}$\
\
$x_{t-1} = \sqrt{\bar{\alpha}_{t-1}}(\frac{x_t-\sqrt{1-\bar{\alpha}_t}\epsilon_\theta}{\sqrt{\bar{\alpha}}_t}) + \sqrt{1-\bar{\alpha}_{t-1}-\sigma^2}\epsilon_\theta + \sigma^2\epsilon$\
\
$并且不是马尔科夫假设，所以不需要严格遵守x_t\to x_{t-1}$\
\
$∴x_{s} = \sqrt{\bar{\alpha}_{s}}(\frac{x_z-\sqrt{1-\bar{\alpha}_z}\epsilon_\theta}{\sqrt{\bar{\alpha}}_z}) + \sqrt{1-\bar{\alpha}_{s}-\sigma^2}\epsilon_\theta + \sigma^2\epsilon$\
\
$s<z，z\le T，s和z不需要连续$\
\
$神奇的是\sigma取任何值，等式都成立$


